In [ ]:
#Created on Mon Mar  9 22:09:02 2020
#
#@author: Ramin Mehdizad Tekiyeh

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import math

In [ ]:
# in this section we read scsvfile and import dataset
df=pd.read_csv("ComputerSpeedData.csv")
# here we calculate average of 4 run time
df["RunF"]=(df["Run1 (ms)"]+df["Run2 (ms)"]+df["Run3 (ms)"]+df["Run4 (ms)"])/4
# here we drop run time columns and insert the average amount into a new column
df=df.drop(columns=['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)'])
df['RunF'].mean()
df['RunF_binary']=np.where(df['RunF']>=100,1,0)
# spliting dataset into train and test
train,test=train_test_split(df,test_size=0.3,random_state=5000)
# extracting values and savingthem in lists
x_train=train.iloc[:,0:14]
y_train=train.iloc[:,-1]
x_test=test.iloc[:,0:14]
y_test=test.iloc[:,-1]

In [ ]:
# this function calculates accuracy scores
def svwork (x_tr,y_tr,x_ts,y_ts,kernel,c):
    # initializing lists
    train_results=[]
    test_results=[]
    train_accuracies=[]
    test_accuracies=[]
    # here wee loop for different classifying models
    for c in c:
        # here we define model
       svc=SVC(kernel=kernel,C=c,gamma='auto')
       # now we train the model
       svc.fit(x_tr,y_tr)
       # here we do the predictions
       train_pred=svc.predict(x_tr)
       test_pred=svc.predict(x_ts) 
       # calculating scores
       fpr,tpr,thresholds=roc_curve(y_tr,train_pred)
       roc_auc=auc(fpr,tpr)
       # Add auc score to previous train results
       train_results.append(roc_auc)
       # performing prediction
       y_pred=svc.predict(x_ts)
       fpr,tpr,thresholds=roc_curve(y_ts,y_pred)
       roc_auc=auc(fpr,tpr)
       # Add auc score to previous test results
       test_results.append(roc_auc)
       # performing train and test accuracy calculation
       train_acc=accuracy_score(y_tr,train_pred)
       test_acc=accuracy_score(y_ts,test_pred)
       # saving results of accuracy in lists
       train_accuracies.append(train_acc)
       test_accuracies.append(test_acc)
    return test_accuracies,train_accuracies,test_results,train_results

In [ ]:
# linear kernel crossvalidationcestiob
print("linear kernel crossvalidation*****",end='\n\n')
c_lr=np.array([10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,10**0])
#  initializing the list
c_logvalues=[]

In [ ]:
# calculatin c values
for c in c_lr:
    c_logvalues.append(math.log10(c))
    
#  initializing the list
acc_score_lr=[]

In [ ]:
# in this loop we calculate cross validation score and save it in a list
for c in c_lr:
    svc=SVC(kernel='linear',C=c,gamma='auto')
    # calculating cross validation score
    scores=cross_val_score(svc,x_train,y_train,cv=5,scoring='accuracy')
    # saving cross validation score in a list
    acc_score_lr.append(scores.mean())
print("acc_score_lr is:       ",acc_score_lr)    

In [ ]:
# now we plot cross validation versus c values
p=acc_score_lr
#gpu linear test vs train
plt.plot(c_logvalues,p,'b',label='Cross Validation Accuracy Score')
plt.legend(loc='best',frameon=False)
plt.ylabel("Accuracy score")
plt.xlabel("log10(c) values")
plt.title("'linear kernel' SVM Cross Validated")
plt.show()

In [ ]:
# in this section we calculate train and test accuracy
kernel='linear'
test_acc_lr,train_acc_lr,test_res_lr,train_res_lr=svwork(x_tr=x_train,y_tr=y_train,x_ts=x_test,y_ts=y_test,kernel=kernel,c=c_lr)

# and here we plot train and test accuracy versus c values
plt.plot(c_logvalues,train_acc_lr,'b',label='Train Accuracy for GPU  Run Time')
plt.plot(c_logvalues,test_acc_lr,'r',label='Test Accuracy for GPU Run Time')
plt.legend(loc='best',frameon=False)
plt.ylabel("Accuracy score")
plt.xlabel("log10(c) values")
plt.title("Train and Test Accuary vs. C values")
plt.show()
